### Import libraries

In [26]:
from IPython.core.display import display
from src.utils.preprocessing import classic_preprocessing,standardize
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

### Load Data

In [3]:
DATA_PATH = '../../data'
X_fine, y_fine = import_data(DATA_PATH, segmentation_type='fine',
                                 drop_user_features=False,
                                 drop_expert=True)

In [23]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='fine',
                                 drop_user_features=False,
                                 drop_expert=False)

In [4]:
display(X_fine.head())
display(y_fine.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               6.0          6.0   
                                     1               3.0          3.0   
                                     2               4.0          4.0   
                                     3               4.0          4.0   
                                     4               4.0          4.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          4.0   
                                     1                3.0          4.0   
                                     2                3.0          4.0   
                                     3                4.0          4.0   
                                     4                5.0          4.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          5.0   
                                     1                4.0          5.0   
                                     2                3.0          3.0   
                                     3                4.0          5.0   
                                     4                4.0          5.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                6.0          6.0   
                                     1                4.0          4.0   
                                     2                4.0          4.0   
                                     3                3.0          4.0   
                                     4                3.0          4.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          6.0  ...   
                                     1                4.0          3.0  ...   
                                     2                4.0          4.0  ...   
                                     3                4.0          3.0  ...   
                                     4                4.0          3.0  ...   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.398299     0.026407   
                                     1           0.230052     0.086242   
                                     2           0.105747     0.180344   
                                     3           0.235517     0.121967   
                                     4           0.230394     0.119689   

                                              PSD_1325-1600  PSD_1600-1900  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.013356       0.033366   
                                     1             0.023806       0.008824   
                                     2             0.034701       0.006258   
                                     3             0.061882       0.002742   
                                     4             0.069418       0.002717   

                                              PSD_2500-2900  PSD_3100-3700  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.043195       0.010385   
                                     1             0.056102       0.011429   
                                     2       

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
                                     2          1.0
                                     3          1.0
                                     4          1.0

### Preprocessing

In [5]:
X_fine = classic_preprocessing(X_fine)

In [24]:
# Save the expert feature for split
expert = X_e.copy().Expert
X_e = classic_preprocessing(X_e)
X_e.Expert = expert.values

### Split expert models

In [27]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

#### 1. Logistic regression

In [6]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  accuracy_score
max_iter oversampling                                         
10000    True          0.402198       0.574983        0.622634
         False         0.157118       0.530696        0.732477
100000   True          0.402198       0.574983        0.622634
         False         0.157118       0.530696        0.732477

In [7]:
# Best model parameters

best_log = log_results.iloc[[log_results.reset_index()['f1_score'].idxmax()]]

display(best_log)


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.402198,0.574983,0.622634


In [28]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=5,
                                       params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=5,
                                       params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=5,
                                       params=LOGISTIC_PARAMS)

In [29]:
# Best model parameters

best_log_1 = log_results_1.iloc[[log_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_log_2 = log_results_2.iloc[[log_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_log_3 = log_results_3.iloc[[log_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_log_1, best_log_2, best_log_3)


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.366321,0.647065,0.736791


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.29471,0.550656,0.645144


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.642027,0.659811,0.660414


#### 2. Linear Discriminant Analysis

In [8]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.402724,0.576162,0.625392
False,0.166116,0.532169,0.731526


In [9]:
# Best model parameters

best_lda = lda_results.iloc[[lda_results.reset_index()['f1_score'].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.402724,0.576162,0.625392


In [30]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=5,
                                       params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=5,
                                       params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=5,
                                       params=LDA_PARAMS)


In [31]:
best_lda_1 = lda_results_1.iloc[[lda_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_lda_2 = lda_results_2.iloc[[lda_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_lda_3 = lda_results_3.iloc[[lda_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.359445,0.643375,0.724473


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.291674,0.54846,0.645144


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.640336,0.659293,0.660138


#### 3. K-nearest Neighbors

In [10]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  accuracy_score
n_neighbors oversampling                                         
1           True          0.538261       0.683665        0.662863
            False         0.515077       0.668555        0.746172
2           True          0.528326       0.675809        0.700238
            False         0.323353       0.586013        0.756633
3           True          0.508429       0.651581        0.584118
            False         0.431194       0.621831        0.739515
4           True          0.510313       0.656332        0.621969
            False         0.290474       0.573358        0.750071
5           True          0.495399       0.634570        0.544746
            False         0.387260       0.605420        0.748264
6           True          0.499344       0.642232        0.578887
            False         0.252190       0.560634        0.745602
7           True          0.485318       0.620226        0.515644
            False         0.330488       0.584059        0.745982
8           True          0.490013       0.629267        0.547599
            False         0.211966       0.547526        0.740086
9           True          0.479705       0.611329        0.494817
            False         0.273073       0.563421        0.739135
10          True          0.480628       0.615864        0.518973
            False         0.187402       0.542011        0.740561
11          True          0.472081       0.599816        0.474085
            False         0.244725       0.556350        0.740846
12          True          0.476706       0.608947        0.500238
            False         0.163097       0.535999        0.739515
13          True          0.466911       0.591658        0.459439
            False         0.212515       0.547146        0.739039
14          True          0.471130       0.600587        0.485402
            False         0.144381       0.530611        0.737233
15          True          0.465740       0.588807        0.450689
            False         0.186899       0.540420        0.737993

In [11]:
# Best model parameters

best_knn = knn_results.iloc[[knn_results.reset_index()['f1_score'].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.538261,0.683665,0.662863


In [32]:
knn_results_1 = hyperparameter_tuning_cv(model='knn', data=X_e_1, labels=y_e_1, cv_k=5, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(model='knn', data=X_e_2, labels=y_e_2, cv_k=5, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(model='knn', data=X_e_3, labels=y_e_3, cv_k=5, params=KNN_PARAMS)

In [33]:
best_knn_1 = knn_results_1.iloc[[knn_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_knn_2 = knn_results_2.iloc[[knn_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_knn_3 = knn_results_3.iloc[[knn_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.46468,0.754597,0.740357


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.466525,0.701827,0.701312


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,False,0.711978,0.733584,0.735448


#### 4. Support Vector Classifier

In [12]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  accuracy_score
kernel gamma oversampling                                         
linear 0.10  True          0.393986       0.572373        0.630147
             False         0.002822       0.500456        0.730480
rbf    0.10  True          0.081298       0.519006        0.738469
             False         0.028483       0.507232        0.734379
linear 0.01  True          0.393986       0.572373        0.630147
             False         0.002822       0.500456        0.730480
rbf    0.01  True          0.421890       0.602247        0.679220
             False         0.067001       0.514689        0.735711

In [13]:
# Best model parameters

best_svc = svc_results.iloc[[svc_results.reset_index()['f1_score'].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.42189,0.602247,0.67922


In [34]:
svc_results_1 = hyperparameter_tuning_cv(model='svc', data=X_e_1, labels=y_e_1, cv_k=5, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(model='svc', data=X_e_2, labels=y_e_2, cv_k=5, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(model='svc', data=X_e_3, labels=y_e_3, cv_k=5, params=SVC_PARAMS)

In [35]:
best_svc_1 = svc_results_1.iloc[[svc_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_svc_2 = svc_results_2.iloc[[svc_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_svc_3 = svc_results_3.iloc[[svc_results_3.reset_index()['roc_auc_score'].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.43055,0.671778,0.824311


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.282456,0.561125,0.740682


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.679652,0.695457,0.696


#### 5. Naive Bayes Classifier

In [14]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.372578,0.559345,0.627199
False,0.335945,0.557551,0.673039


In [15]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()['f1_score'].idxmax()]]

display(best_nb)


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.372578,0.559345,0.627199


In [36]:
nb_results_1 = hyperparameter_tuning_cv(model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=5, params=NAIVE_BAYES_PARAMS)

In [37]:
best_nb_1 = nb_results_1.iloc[[nb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_nb_2 = nb_results_2.iloc[[nb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_nb_3 = nb_results_3.iloc[[nb_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.346586,0.639599,0.684603


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
False,0.220341,0.523959,0.71916


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.47421,0.589975,0.603034


#### 6. Decision Tree

In [16]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  accuracy_score
max_depth oversampling                                         
3         True          0.352358       0.540014        0.582406
          False         0.079716       0.512296        0.727722
5         True          0.324248       0.545553        0.650309
          False         0.161518       0.529254        0.726676
7         True          0.354180       0.561142        0.653543
          False         0.245347       0.549321        0.725725

In [17]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()['f1_score'].idxmax()]]

display(best_dt)


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
7,True,0.35418,0.561142,0.653543


In [38]:
dt_results_1 = hyperparameter_tuning_cv(model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=5, params=DECISION_TREE_PARAMS)

In [39]:
best_dt_1 = dt_results_1.iloc[[dt_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_dt_2 = dt_results_2.iloc[[dt_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_dt_3 = dt_results_3.iloc[[dt_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
5,True,0.352647,0.639624,0.719935


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
7,True,0.28648,0.549362,0.665617


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
5,True,0.614093,0.619935,0.616


#### 7. Random Forest

In [18]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.371815       0.551211        0.602187
                       False         0.035301       0.505504        0.730005
          5            True          0.397162       0.564723        0.597527
                       False         0.026420       0.505037        0.731621
          7            True          0.389541       0.558836        0.595055
                       False         0.009011       0.501240        0.730195
5         3            True          0.362799       0.552028        0.622825
                       False         0.088477       0.513384        0.727342
          5            True          0.382174       0.565954        0.629767
                       False         0.080113       0.514067        0.730861
          7            True          0.391091       0.573344        0.639372
                       False         0.054063       0.509356        0.730956
7         3            True          0.376188       0.563850        0.633571
                       False         0.147680       0.522385        0.721446
          5            True          0.374291       0.566337        0.646790
                       False         0.116288       0.520445        0.729815
          7            True          0.380217       0.574219        0.659439
                       False         0.086110       0.515306        0.731051

In [19]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()['f1_score'].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,5,True,0.397162,0.564723,0.597527


In [40]:
rf_results_1 = hyperparameter_tuning_cv(model='random_forest', data=X_e_1, labels=y_e_1, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(model='random_forest', data=X_e_2, labels=y_e_2, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(model='random_forest', data=X_e_3, labels=y_e_3, cv_k=5, params=RANDOM_FOREST_PARAMS)


In [41]:
best_rf_1 = rf_results_1.iloc[[rf_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_rf_2 = rf_results_2.iloc[[rf_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_rf_3 = rf_results_3.iloc[[rf_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,5,True,0.36226,0.652259,0.70081


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,7,True,0.288521,0.559309,0.71601


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,True,0.614339,0.640894,0.643034


#### 8. Gradient Boosting

In [20]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.373577       0.563819        0.639372
                       False         0.000000       0.500000        0.730480
          5            True          0.371229       0.567133        0.650214
                       False         0.000677       0.500169        0.730575
          7            True          0.360229       0.561771        0.652211
                       False         0.002106       0.500527        0.730766
5         3            True          0.344576       0.569227        0.693485
                       False         0.000000       0.500000        0.730480
          5            True          0.334424       0.569090        0.703947
                       False         0.004197       0.500921        0.730861
          7            True          0.337949       0.572857        0.709938
                       False         0.018032       0.503728        0.731717
7         3            True          0.356986       0.568124        0.675892
                       False         0.000690       0.499979        0.730290
          5            True          0.347414       0.573140        0.700048
                       False         0.031162       0.506687        0.732858
          7            True          0.349171       0.576724        0.708607
                       False         0.068912       0.514736        0.735140

In [21]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()['f1_score'].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,3,True,0.373577,0.563819,0.639372


In [42]:
gb_results_1 = hyperparameter_tuning_cv(model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)

In [43]:
best_gb_1 = gb_results_1.iloc[[gb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_gb_2 = gb_results_2.iloc[[gb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_gb_3 = gb_results_3.iloc[[gb_results_3.reset_index()['roc_auc_score'].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,True,0.412485,0.675197,0.783468


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,7,True,0.279849,0.563229,0.75853


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,False,0.631032,0.681612,0.688


### Results

In [22]:
display('logistic', best_log)
display('lda', best_lda)
display('knn', best_knn)
display('svc', best_svc)
display('naive_bayes', best_nb)
display('decision_tree', best_dt)
display('random_forest', best_rf)
display('gradient_boosting', best_gb)

'logistic'

,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.402198,0.574983,0.622634


'lda'

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.402724,0.576162,0.625392


'knn'

...

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
7,True,0.35418,0.561142,0.653543


'random_forest'

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,5,True,0.397162,0.564723,0.597527


'gradient_boosting'

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,3,True,0.373577,0.563819,0.639372


In [44]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1,best_knn_2,best_knn_3]))
display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'logistic'

f1_score  roc_auc_score  accuracy_score
max_iter oversampling                                         
10000    True          0.366321       0.647065        0.736791
         True          0.294710       0.550656        0.645144
         True          0.642027       0.659811        0.660414

'lda'

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.359445,0.643375,0.724473
True,0.291674,0.548460,0.645144
True,0.640336,0.659293,0.660138


'knn'

...

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
5,True,0.352647,0.639624,0.719935
7,True,0.286480,0.549362,0.665617
5,True,0.614093,0.619935,0.616000


'random_forest'

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,5,True,0.362260,0.652259,0.700810
7,7,True,0.288521,0.559309,0.716010
5,7,True,0.614339,0.640894,0.643034


'gradient_boosting'

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,True,0.412485,0.675197,0.783468
7,7,True,0.279849,0.563229,0.758530
5,7,False,0.631032,0.681612,0.688000


### Conclusions

TODO